In [1]:
import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import seaborn

In [2]:
airbnb_raw = pd.read_csv(r'.\listings.csv')
airbnb_location = pd.DataFrame(airbnb_raw, columns = ['latitude', 'longitude','neighbourhood']) 

airbnb_location['lat'] =  pd.to_numeric(airbnb_location['latitude'], downcast="float")
airbnb_location['lon'] = pd.to_numeric(airbnb_location['longitude'], downcast="float")
airbnb_location['neighbourhood'] = airbnb_raw['neighbourhood_cleansed']
van_airbnb = airbnb_location[(airbnb_location['lat']<=49.5) &
                                                (airbnb_location['lat']>=49)& 
                                                (airbnb_location['lon']<= -122)&
                                                (airbnb_location['lon']>= -123.5)]

In [11]:
fig = px.scatter_mapbox(airbnb_location,
                    lat=airbnb_location.latitude,
                    lon=airbnb_location.longitude,
                    color=airbnb_location.neighbourhood, color_continuous_scale=px.colors.cyclical.IceFire
                    ,size_max=10, zoom=11)
fig.update_layout(mapbox_style="open-street-map",
                 margin = {'l':0, 'r':0, 'b':0, 't':0})
fig.show()

In [4]:
fig = px.scatter_mapbox(van_airbnb,
                    lat=van_airbnb.latitude,
                    lon=van_airbnb.longitude,
                    color=van_airbnb.neighbourhood, color_continuous_scale=px.colors.cyclical.IceFire
                    ,size_max=10, zoom=11)
fig.update_layout(mapbox_style="open-street-map",
                 margin = {'l':0, 'r':0, 'b':0, 't':0})
fig.show()

In [5]:
# amenities = pd.read_json('osm/amenities-vancouver.json.gz', lines=True)
amenities = pd.read_csv('interesting_amenities.csv')
# amenities

In [6]:
# https://stackoverflow.com/a/21623206
def distance_count(airbnb, amenities):
    lat1 = airbnb['lat']
    lat2 = amenities['lat']
    lon1 = airbnb['lon']
    lon2 = amenities['lon']
    p = np.pi/180
    b = np.cos((lat2.sub(lat1)).mul(p)).div(2) #series
    c = np.cos(lat1*p) #number
    d = np.cos(lat2.mul(p)) #series
    e = np.subtract(1,np.cos((lon2.sub(lon1)).mul(p))).div(2) #series
    a = np.subtract(0.5, b).add((d.mul(c)).mul(e))
    
    distances = np.arcsin(np.sqrt(a)).mul(12742)
    
    nearby_distances = distances[distances < 0.1]
    count = len(nearby_distances.index)
    return count
# https://stackoverflow.com/a/21623206


In [7]:
airbnb_raw['num_of_amenities'] = airbnb_location.apply(lambda x: distance_count(x, amenities), axis=1)
# airbnb_raw

In [8]:
airbnb_filterd = airbnb_raw[airbnb_raw['num_of_amenities']>30]
airbnb_filterd

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,...,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,num_of_amenities
16,26047,https://www.airbnb.com/rooms/26047,20201016174011,2020-10-17,The Alpha Downtown Accommodation by COMFYSUITES,This lovely 1-BD condo is your private oasis i...,Central Downtown location. 2 blocks away from ...,https://a0.muscache.com/pictures/0ead6a47-5af1...,109845,https://www.airbnb.com/users/show/109845,...,10.0,9.0,NaN,t,4,4,0,0,0.30,33
20,42645,https://www.airbnb.com/rooms/42645,20201016174011,2020-10-17,The Charlie by COMFYSUITES,Our lovely 1-Bedroom comfortable private apart...,Central Downtown location. 2 blocks away from ...,https://a0.muscache.com/pictures/cfbb44d3-5671...,109845,https://www.airbnb.com/users/show/109845,...,10.0,9.0,NaN,t,4,4,0,0,0.23,32
23,54261,https://www.airbnb.com/rooms/54261,20201016174011,2020-10-17,Yaletown 1 Bedroom Condo,"<b>The space</b><br />Corner suite condo, in t...",NaN,https://a0.muscache.com/pictures/2265827/8d120...,254337,https://www.airbnb.com/users/show/254337,...,10.0,9.0,NaN,f,2,2,0,0,0.32,33
26,73841,https://www.airbnb.com/rooms/73841,20201016174011,2020-10-17,Amazing Downtown Vancouver Condo!,A fantastic home base to explore Vancouver and...,"The Brava is a 14-year old building, very pet ...",https://a0.muscache.com/pictures/4d09315e-ab7c...,386761,https://www.airbnb.com/users/show/386761,...,10.0,10.0,NaN,f,1,1,0,0,0.12,32
49,258636,https://www.airbnb.com/rooms/258636,20201016174011,2020-10-25,FABULOUS Luxury Downtown Vancouver!,<b>The space</b><br />Come and enjoy this lux...,NaN,https://a0.muscache.com/pictures/2822389/1ceae...,948852,https://www.airbnb.com/users/show/948852,...,10.0,9.0,NaN,f,1,1,0,0,0.48,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4699,45767623,https://www.airbnb.com/rooms/45767623,20201016174011,2020-10-17,2 Bedroom with an Awesome View on Abbott,This unit is in the heart of Downtown Vancouve...,NaN,https://a0.muscache.com/pictures/635695f0-ee39...,30899,https://www.airbnb.com/users/show/30899,...,NaN,NaN,NaN,t,46,46,0,0,NaN,31
4701,45771785,https://www.airbnb.com/rooms/45771785,20201016174011,2020-10-25,Budget 1Br next to Arena+ free parking,You’ll be staying in 1 bedroom + 1 den apartme...,NaN,https://a0.muscache.com/pictures/ad811b01-60ae...,286554125,https://www.airbnb.com/users/show/286554125,...,10.0,10.0,Exempt,t,1,1,0,0,2.00,31
4704,45784656,https://www.airbnb.com/rooms/45784656,20201016174011,2020-10-17,Spacious Downtown Vancouver Studio w/h Murphy ...,"Situated right between Gastown and Chinatown, ...",NaN,https://a0.muscache.com/pictures/62154757-62e3...,18175645,https://www.airbnb.com/users/show/18175645,...,NaN,NaN,20-301471,f,1,1,0,0,NaN,42
4708,45802041,https://www.airbnb.com/rooms/45802041,20201016174011,2020-10-17,Spacious Get-away Fun House at Granville,Have fun with us at our Fun house! Industrial ...,"Located in the heart of Vancouver, Granville S...",https://a0.muscache.com/pictures/miso/Hosting-...,371298497,https://www.airbnb.com/users/show/371298497,...,10.0,10.0,20-462112,t,1,1,0,0,2.00,45


In [9]:
fig = px.scatter_mapbox(airbnb_filterd,
                    lat=airbnb_filterd.latitude,
                    lon=airbnb_filterd.longitude,
                    color=airbnb_filterd.num_of_amenities
                    ,size_max=10, zoom=12)
fig.update_layout(mapbox_style="open-street-map",
                 margin = {'l':0, 'r':0, 'b':0, 't':0})
fig.show()